In [ ]:
%pip install langchain langchain_community langchain-openai langchain-ollama faiss-cpu

In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Đường dẫn tới thư mục chứa các file .txt
folder_path = "TuyenSinhPTITData"

# Danh sách các file .txt trong thư mục
txt_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".txt")]

# Hàm để load nội dung từ file .txt
def load_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Load nội dung từ tất cả các file .txt
docs = [load_txt(file) for file in txt_files]
doc_objects = [Document(page_content=doc, metadata={"source": file}) for doc, file in zip(docs, txt_files)]

# Initialize a text splitter with specified chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=256, chunk_overlap=20
)
doc_splits = text_splitter.split_documents(doc_objects)


In [2]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Khởi tạo mô hình SentenceTransformers
model_name = 'bkai-foundation-models/vietnamese-bi-encoder'
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Tạo FAISS Vector Store
vectorstore = FAISS.from_documents(doc_splits, embedding=embeddings)
# Chuyển đổi vectorstore thành retriever
retriever = vectorstore.as_retriever(k=4)


d:\Code\notebook\RagOllama\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Huy10\AppData\Local\Temp\ipykernel_15824\583171914.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)


In [3]:
vectorstore.save_local("faiss_index")

In [4]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Khởi tạo mô hình SentenceTransformers
model_name = 'bkai-foundation-models/vietnamese-bi-encoder'
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Tạo FAISS Vector Store
vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
# Chuyển đổi vectorstore thành retriever
retriever = vectorstore.as_retriever(k=4)

In [3]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
# Tạo Prompt hương dẫn cho mô hình
prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks.
    Use the following documents to answer the question.
    If you don't know the answer, just say that you don't know:
    Question: {question}
    Documents: {documents}
    Answer:
    """,
    input_variables=["question", "documents"],
)

llm = ChatOllama(
    model="llama3.1",
    temperature=0,
)

rag_chain = prompt | llm | StrOutputParser()

In [4]:
# Khai báo RAG application class
class RAGApplication:
    def __init__(self, retriever, rag_chain):
        self.retriever = retriever
        self.rag_chain = rag_chain
    def run(self, question):
        # Truy xuất dữ liệu
        documents = self.retriever.invoke(question)
        # Trích xuất nội dung từ document
        doc_texts = "\\n".join([doc.page_content for doc in documents])
        # Sinh câu trả lời từ câu hỏi và nội dung văn bản
        answer = self.rag_chain.invoke({"question": question, "documents": doc_texts})
        return answer

In [8]:

rag_application = RAGApplication(retriever, rag_chain)
# Example usage
question = "Cơ hội việc làm khi tốt nghiệp ngành Quản trị kinh doang"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: Cơ hội việc làm khi tốt nghiệp ngành Quản trị kinh doang
Answer: Sinh viên tốt nghiệp ngành Quản trị kinh doanh có thể đảm nhận các vị trí công việc như quản lý, cán bộ nghiên cứu, giảng dạy về marketing tại các Viện, trường nghề nghiệp. Họ cũng có thể tự tạo lập doanh nghiệp (start up) và tìm kiếm cơ hội kinh doanh riêng. Ngoài ra, sinh viên này còn có thể làm việc tại các vị trí cụ thể khác trong lĩnh vực kinh doanh. Có thể tiếp tục học lên trình độ sau đại học ngành Kinh doanh và quản lý.


In [5]:
import gradio as gr

rag_application = RAGApplication(retriever, rag_chain)

def generate(messages, history):
    return rag_application.run(messages["text"])

demo = gr.ChatInterface(
    fn=generate,
    type="messages",
     examples=[
        ["Điểm chuẩn ngành công nghệ thông tin năm 2023?"],
        ["Cơ sở vật chất của học viện như thế nào?"],
        ["Các phương thức tuyển sinh đại học chính quy?"],
    ],
    title="Chatbot tư vấn tuyển sinh Học viện Công nghệ Bưu chính Viễn thông năm 2024",
    multimodal=True,
)
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
